In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sentiment_analyzer
import csv

/home/charles/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load the df that we created in the other notebook
df_files = pd.read_csv("safe/df_file_final.csv", lineterminator='\n')

In [3]:
## Transform into array from string. Also rename column if a problem occured during loading.
from ast import literal_eval
if("sentences\r" in df_files.columns):
    df_files = df_files.rename(index=str,columns={'sentences\r': 'sentences'})
df_files.sentences = df_files.sentences.apply(literal_eval)

In [4]:
def average(tupple, sentimentAnalyzer):
    """perform a weighted average using the importance of each sentence as a weight"""
    sents, imporances = zip(*tupple)
    sents = list(sents)
    importance = list(imporances)
    pred = np.squeeze(sentimentAnalyzer.predict(sents))
    return np.average(pred, weights=imporances)

def find_sentiment(important_sents,sentimentAnalyzer):
    """Return sentiment final given top_k phrases and company_phrases, with a hyper parameter to sum to a final result."""
    try:
        if important_sents == [-500]:
            return -500
        interest_sent, top_sent = important_sents
        lambda_ = 0.3
        return (1-lambda_)*average(interest_sent,sentimentAnalyzer) + \
            lambda_*average(top_sent, sentimentAnalyzer)
    except:
        return -999

In [5]:
#model = keras.models.load_model('models/my_amazon_model.h5')
analyzser = sentiment_analyzer.SentimentAnalyzer('models/my_model.h5')
df_files.sentiment = df_files.apply(lambda row: np.mean(find_sentiment(row['sentences'],analyzser)),axis=1)

Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


100%|██████████| 5/5 [00:00<00:00, 208.78it/s]


100%|██████████| 5/5 [00:00<00:00, 182.54it/s]


In [6]:
df_files.head()

,Unnamed: 0,Unnamed: 0.1,id,filename,company,file_path,sentiment,sentences
0,0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,0.631471,([(Keppel Capital aims to create value and del...
1,1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,0.660154,([(Company Backg round Keppel is a diversified...
2,2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,0.547001,([(The new SOCAR-Keppel Shipyard will be devel...
3,3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,-999.000000,[-500]
4,4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,0.621223,([(Jurong Shipyard Pte Ltd (Songa Eclipse) Jur...


In [7]:
##Get the index of the top and worst document from a dataframe with value filled.
def getSubmission(df):
    """Return : 4 array of index that correspond to worst and best document of each company"""
    dfCleaned = df[df['sentiment']>0.01]
    top_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=False).head(10).id)
    down_keppel = list(dfCleaned[dfCleaned['company'] == 'Keppel'].sort_values('sentiment',ascending=True).head(10).id)
    top_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=False).head(10).id)
    down_prud = list(dfCleaned[dfCleaned['company'] == 'Prudential'].sort_values('sentiment',ascending=True).head(10).id)
    return top_keppel,down_keppel, top_prud, down_prud

In [8]:
def createSubCSV(topId_k,downId_k,topId_p,downId_p):
    """Create a CSV submission from the index of top/worst document of each company"""
    with open("sub.csv",'w',newline='') as csvfile:
        spamwritter = csv.writer(csvfile)
        spamwritter.writerow(['id','rank'])
        for i in range(len(df_files.sentiment)):
            if(i in downId_k):
                spamwritter.writerow([i,(downId_k.index(i)-10)])
            elif(i in topId_k):
                spamwritter.writerow([i,(10-topId_k.index(i))])
            elif(i in downId_p):
                spamwritter.writerow([i,(downId_p.index(i)-10)])
            elif(i in topId_p):
                spamwritter.writerow([i,(10-topId_p.index(i))])
            else:
                spamwritter.writerow([i,0])

In [9]:
top_k,down_k, top_p, down_p = getSubmission(df_files)
createSubCSV(top_k,down_k, top_p, down_p)